In [1]:
#!pip install selenium
#!pip install cloudscraper

In [68]:
from selenium import webdriver
from tqdm.notebook import tqdm
import cloudscraper
import pandas as pd
import requests
import os
from PIL import Image
from io import BytesIO
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [69]:
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=chrome_options)

driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
'source': '''
delete window.cdc_adoQpoasnfa76pfcZLmcfl_Array;
delete window.cdc_adoQpoasnfa76pfcZLmcfl_Promise;
delete window.cdc_adoQpoasnfa76pfcZLmcfl_Symbol;
delete window.cdc_adoQpoasnfa76pfcZLmcfl_Object;
delete window.cdc_adoQpoasnfa76pfcZLmcfl_Proxy;
'''})

{'identifier': '2'}

In [70]:
url = 'https://www.dawlance.com.pk/'
driver.get(url)
menu_links = driver.find_elements(By.CLASS_NAME, 'lv1-li')

categories = []
subcategories = []

for link in menu_links:
    categories.append(link.find_element(By.TAG_NAME, 'a').get_attribute('href'))
    
for cat in categories:
    driver.get(cat)
    subcat_ul = driver.find_element(By.CLASS_NAME, 'category-list')
    subcat_urls = [x.get_attribute('href') for x in subcat_ul.find_elements(By.TAG_NAME, 'a')]
    
    subcategories.extend(subcat_urls)

len(subcategories)

23

In [71]:
products = []
categories = []
clean_image_urls = []
image_names = []

for cat_index, category in tqdm(enumerate(subcategories)):
    
    driver.get(category)
    
    category_name = driver.find_element(By.TAG_NAME, 'h1').text
    
    # Find item cards 
    divs_items = driver.find_elements(By.CLASS_NAME, "product-item")

    for good_index, item_card in enumerate(divs_items):
        
        product_name_1 = item_card.find_element(By.CLASS_NAME, "t").text
        product_name_2 = item_card.find_element(By.CLASS_NAME, "v").text 
        product_name = product_name_1 + " " + product_name_2 + " made by Dawlance"

        images = item_card.find_elements(By.CLASS_NAME, "lazy")

        src_sets = [x.get_attribute("data-srcset") for x in images]
        
        if len(src_sets) > 0:
            image_urls = src_sets[0].split(",")
        else:
            continue
        
        for img_index, img_url in enumerate(src_sets):
            parts = img_url.split("/")
            original_path = "/".join(parts[:4]) + "/1000Wx1000H/image.webp"
            image_name = "dawlance_0000" + str(cat_index) + "_" + str(good_index) + "_" + str(img_index) + ".png"
            
            clean_image_urls.append("https://www.dawlance.com.pk" + original_path)
            products.append(product_name)
            categories.append(category_name)
            image_names.append(image_name)
    time.sleep(2)

0it [00:00, ?it/s]

In [72]:
df = pd.DataFrame({'category':categories, 
                    'text':products, 
                    'img_urls': clean_image_urls,
                    'image_name':image_names}) 

In [74]:
df.to_csv("img_urls.csv", index=False)

In [78]:
wrong_indexes = []

for index, (img, image_url) in enumerate(zip(df['image_name'], df['img_urls'])):   
    
    image_path = "images_dawlance/" + img
    
    # Download image and save to folder
    driver.get(image_url)
    try:
        image = driver.find_element(By.TAG_NAME, 'img')
    except:
        wrong_indexes.append(index)
        continue
        
    driver.execute_script("arguments[0].style.backgroundColor = 'hsl(0 0% 100% / 1)';", image)
    driver.execute_script("document.body.style.backgroundColor = 'white';")
    time.sleep(1)
    image.screenshot(image_path)
    time.sleep(2)

In [79]:
image_files = set(os.listdir("images_dawlance"))
len(image_files)

546

In [80]:
df_filtered = df[df['image_name'].isin(image_files)]
df_filtered.shape

(545, 4)

In [81]:
df_filtered[['category','text','image_name']].to_csv("dawlence.csv", index=False)